In [1]:
import os
os.chdir("src")
from utils import get_model

/home/doluk/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
from types import SimpleNamespace
from root_dir_path import ROOT_DIR
from peft import get_peft_model
from utils import load_data
from encode import train
from tqdm import tqdm
args = SimpleNamespace()
args.model_name = "qwen2.5-1.5b-instruct"
args.lora_rank = 2
args.lora_alpha = 32
args.dataset = "popqa"
args.data_type = "total"
args.augment_model = args.model_name
args.learning_rate = 3e-4
args.sample=1
args.num_train_epochs=1 
args.with_cot=False
args.per_device_train_batch_size=1
data_list = load_data(args.dataset, args.data_type, args.augment_model)
cot_name = "cot" if args.with_cot else "direct"

In [4]:
model, tokenizer, _generation_config = get_model(args.model_name)

In [16]:
init_adapter_path = os.path.join(
            ROOT_DIR, 
            "warmup/lora_base_weight", 
            args.model_name, 
            "direct"
        )

os.path.exists(os.path.join(init_adapter_path, "adapter_model.safetensors"))

True

In [38]:
for filename, fulldata in data_list:
    filename = filename.split('.')[0] 
    print(f"### Solving {filename} ###")
    output_dir = os.path.join(
        ROOT_DIR, 
        "offline", 
        "warm_up",
        args.model_name, 
        f"rank={args.lora_rank}_alpha={args.lora_alpha}",
        args.dataset,
        f"lr={args.learning_rate}_epoch={args.num_train_epochs}_{cot_name}",
        f"aug_model={args.augment_model}",
        filename,
    )
    os.makedirs(output_dir, exist_ok=True)
    fulldata = fulldata if args.sample == -1 else fulldata[:args.sample]
    for did, data in tqdm(enumerate(fulldata), total=len(fulldata)):
        augment = data["augment"]
        for pid in range(len(augment)):
            save_path = os.path.join(output_dir, f"data_{did}", f"passage_{pid}")
            if os.path.exists(os.path.join(save_path, "adapter_model.safetensors")):
                continue
            model = train(data["question"], [augment[pid]], args, model, tokenizer, 
                        init_adapter_path, save_path)     
            break
        break
    break

### Solving total ###


  0%|                                                                                                                    | 0/1 [00:06<?, ?it/s]
